In [1]:
from table import HoldemTable, OmahaTable
import hand


from exceptions import *
from utils import *
from hand import Hand
from ranker import *

In [2]:
num = ["2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K", "A"]
suit = ["d", "c", "s", "h"]
a = [n + s for n in num for s in suit]
hand_ranking_order = ["High Card", "One Pair", "Two Pairs", "Three of a Kind", "Straight", "Flush", "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]


In [3]:

def give_odds(flop, turn, river, p1_hand, p2_hand):
    #flop cards and calcs
    game = HoldemTable(num_players= 2, deck_type='full')
    game.add_to_hand(1, p1_hand)
    game.add_to_hand(2, p2_hand)

    game.add_to_community(flop)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    flop_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))
    
    #turn cards and calcs
    game.add_to_community(turn)
    community_cards, undrawn_combos = game.simulation_preparation(150000)
    turn_strength = game.hand_strength_analysis(res_arr = game.simulate_calculation(community_cards, undrawn_combos))

    #river cards and calcs
    game.add_to_community(river)
    who_won, what_hand = winner(game)
    return flop_strength, turn_strength, who_won, what_hand

def winner(self):
    player_rank = np.zeros(self.num_players, dtype=np.int64)
    player_hand_type = np.zeros(self.num_players, dtype=np.int64)

    for player in range(self.num_players):
        player_combos, player_res_arr = self.player_hands[player + 1].hand_value(self.community_arr)
        player_rank[player] = np.max(player_res_arr)
        player_hand_type[player] = np.max(player_res_arr) // 16 ** 5

    if (np.max(player_rank) == player_rank).sum() == 1:
        if np.argmax(player_rank) + 1 == 1:
            return np.argmax(player_rank) + 1, hand_type_dict[player_hand_type[np.argmax(player_rank)]]
        else:
            return 0, hand_type_dict[player_hand_type[np.argmax(player_rank)]]
    else:
        winners, = np.where(np.max(player_rank) == player_rank)
        return 0.5, hand_type_dict[player_hand_type[winners[0]]]

In [4]:


def find_vectors(flop_odds, turn_odds, winning_player, winning_hand):
    flop_vector = [0] * len(hand_ranking_order)
    for hand, probability in flop_odds[1].items():
        if hand in hand_ranking_order:
            index = hand_ranking_order.index(hand)
            flop_vector[index] = probability 

    #Initialize odds vector with 0 for all possible hands
    odds_vector = [0] * len(hand_ranking_order)

    #Assuming turn_odds[1] is a dictionary mapping hand ranking names to their odds
    for hand, probability in turn_odds[1].items():
        if hand in hand_ranking_order:
            index = hand_ranking_order.index(hand)
            odds_vector[index] = probability 

    #winning player vector, with index of winning hand set to 1
    win_vector = [0] * (len(hand_ranking_order) + 1)
    if winning_hand in hand_ranking_order:
        index = hand_ranking_order.index(hand)
        win_vector[index] = 1
    #last column gives 1 if player 1 wins, 0.5 if tie, and 0 if player 2 wins
    win_vector[-1] = winning_player
    return flop_vector, odds_vector, win_vector



In [19]:
from itertools import combinations

#Example usage
p1hand = ['Td', 'Ts']
flop = ['2d', '3d', '4d']
turn = ['5d']
river = ['6d']
# Remove cards already in play from the remaining cards
cards_in_play = set(p1hand + flop + turn + river)
filtered_remaining_cards = [card for card in remaining_cards if card not in cards_in_play]

# Generate all possible hands for player 2 from the filtered remaining cards
p2hands = list(combinations(filtered_remaining_cards, 2))

for hand in p2hands:
    flop_odds, turn_odds, winning_player, winning_hand = give_odds(flop, turn, river, p1hand, hand)
    flop_final, odds_final, win_final = find_vectors(flop_odds, turn_odds, winning_player, winning_hand)
    print(f"Player 2 Hand: {hand}")

Player 2 Hand: ('2c', '2s')
Player 2 Hand: ('2c', '2h')
Player 2 Hand: ('2c', '3c')
Player 2 Hand: ('2c', '3s')
Player 2 Hand: ('2c', '3h')
Player 2 Hand: ('2c', '4c')
Player 2 Hand: ('2c', '4s')
Player 2 Hand: ('2c', '4h')
Player 2 Hand: ('2c', '5c')
Player 2 Hand: ('2c', '5s')
Player 2 Hand: ('2c', '5h')
Player 2 Hand: ('2c', '6c')
Player 2 Hand: ('2c', '6s')
Player 2 Hand: ('2c', '6h')
Player 2 Hand: ('2c', '7d')
Player 2 Hand: ('2c', '7c')
Player 2 Hand: ('2c', '7s')
Player 2 Hand: ('2c', '7h')
Player 2 Hand: ('2c', '8d')
Player 2 Hand: ('2c', '8c')
Player 2 Hand: ('2c', '8s')
Player 2 Hand: ('2c', '8h')
Player 2 Hand: ('2c', '9d')
Player 2 Hand: ('2c', '9c')
Player 2 Hand: ('2c', '9s')
Player 2 Hand: ('2c', '9h')
Player 2 Hand: ('2c', 'Tc')
Player 2 Hand: ('2c', 'Th')
Player 2 Hand: ('2c', 'Jd')
Player 2 Hand: ('2c', 'Jc')
Player 2 Hand: ('2c', 'Js')
Player 2 Hand: ('2c', 'Jh')
Player 2 Hand: ('2c', 'Qd')
Player 2 Hand: ('2c', 'Qc')
Player 2 Hand: ('2c', 'Qs')
Player 2 Hand: ('2c'

DeckException: Card 2c is not in the Deck

Problems: 
 - VERY IMPORTANT: there is an issue with recognizing higher hands within each bracket, i.e., king high flush vs ace high flush. There is a reweighing formula, however would be difficult to implement
    - Below is Chat's response
 - Trim this whole things down to be more computationally effective
 - compute every single permutation, roughly 4.5 trillion 
 - how to get into csv

In [6]:
def get_hand_weight(hand_type: str, highest_card: int) -> float:
    """
    Calculate the weight of a poker hand based on its type and highest card.

    Parameters:
    hand_type (str): The type of poker hand (e.g., "Royal Flush", "Straight Flush", etc.).
    highest_card (int): The highest card in the hand (2-14, where 14 represents an Ace).

    Returns:
    float: The weighted strength of the poker hand.
    """
    base_weights = {
        "Royal Flush": 1.00,
        "Straight Flush": 0.95,
        "Four of a Kind": 0.90,
        "Full House": 0.80,
        "Flush": 0.70,
        "Straight": 0.60,
        "Three of a Kind": 0.50,
        "Two Pair": 0.35,
        "One Pair": 0.20,
        "High Card": 0.10,
    }

    if hand_type not in base_weights:
        raise ValueError("Invalid hand type")

    base_weight = base_weights[hand_type]
    rank_factor = 0.8 + 0.2 * (highest_card / 14)

    return round(base_weight * rank_factor, 3)

#Example usage:
print(get_hand_weight("Straight Flush", 5))   # Lower Straight Flush
print(get_hand_weight("Straight Flush", 13))  # Higher Straight Flush 

0.828
0.936
